In [2]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import holidays  # pip install holidays

# ----------------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------------
RAW_PATH   = pathlib.Path("../api_data/hourly_building_consumption.csv")
LOOK_BACK  = 72     # 3 days of history (hours)
N_FORECAST = 24     # forecast next 24 hours
EPOCHS     = 80
BATCH_SIZE = 256
PATIENCE   = 8      # early stopping
TARGET_COL = "Total_consumption"

FEATURE_COLS = [
    "Total_consumption", "hour_sin", "hour_cos",
    "dow_sin", "dow_cos", "lag1", "lag10", "lag50",
    "roll1", "roll10", "roll50", "time_delta",
    "is_month_start", "is_month_end", "is_holiday"
]

# ----------------------------------------------------------
# HELPER: add time and calendar features
# ----------------------------------------------------------
def add_features(df):
    df = df.copy()
    df['hour']        = df.index.hour
    df['day_of_week'] = df.index.dayofweek

    # cyclic encodings
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['dow_sin']  = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['dow_cos']  = np.cos(2 * np.pi * df['day_of_week'] / 7)

    # lags & rolling stats
    df['lag1']  = df[TARGET_COL].shift(1)
    df['lag10'] = df[TARGET_COL].shift(10)
    df['lag50'] = df[TARGET_COL].shift(50)
    df['roll1']  = df[TARGET_COL].shift(1).rolling(1, min_periods=1).mean()
    df['roll10'] = df[TARGET_COL].shift(1).rolling(10, min_periods=1).mean()
    df['roll50'] = df[TARGET_COL].shift(1).rolling(50, min_periods=1).mean()

    # time delta (hours)
    df['time_delta'] = df.index.to_series().diff().dt.total_seconds().div(3600)

    # calendar flags
    df['is_month_start'] = df.index.is_month_start.astype(int)
    df['is_month_end']   = df.index.is_month_end.astype(int)

    # holiday flag (Swiss)
    swiss_hols = holidays.Switzerland()
    df['is_holiday'] = df.index.normalize().isin(swiss_hols).astype(int)

    return df.dropna()

# ----------------------------------------------------------
# HELPER: create sliding windows
# ----------------------------------------------------------
def make_sequences(df, features, seq_len, n_forecast):
    arr = df[features].values
    X, y = [], []
    for i in range(seq_len, len(arr) - n_forecast + 1):
        X.append(arr[i-seq_len:i])       # (seq_len, n_features)
        y.append(arr[i:i+n_forecast, 0]) # target is first col
    return np.array(X), np.array(y)

# ----------------------------------------------------------
# LOAD & PREP
# ----------------------------------------------------------
df = pd.read_csv(RAW_PATH, parse_dates=['Hour'], index_col='Hour').sort_index()
# filter to workdays/hours if needed; else use full df
df = df.copy()

# add features and drop NaNs
df_feat = add_features(df)

# scale features and target together
scaler = MinMaxScaler()
df_scaled = df_feat.copy()
df_scaled[[TARGET_COL] + [c for c in FEATURE_COLS if c != TARGET_COL]] = scaler.fit_transform(
    df_feat[[TARGET_COL] + [c for c in FEATURE_COLS if c != TARGET_COL]]
)

# ----------------------------------------------------------
# CREATE SEQUENCES & SPLIT LAST DAY
# ----------------------------------------------------------
X, y = make_sequences(df_scaled, FEATURE_COLS, LOOK_BACK, N_FORECAST)
# use last sequence as forecast
X_train, y_train = X[:-1], y[:-1]
X_last,  y_last  = X[-1:], y[-1:]

# ----------------------------------------------------------
# BUILD & TRAIN LSTM
# ----------------------------------------------------------
n_features = X.shape[2]
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(LOOK_BACK, n_features)),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(N_FORECAST)
])
model.compile(optimizer='adam', loss='mse')

callbacks = [EarlyStopping(patience=PATIENCE, restore_best_weights=True)]
model.fit(
    X_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=0.1,
    callbacks=callbacks,
    verbose=2
)

# ----------------------------------------------------------
# FORECAST LAST DAY & EVALUATE
# ----------------------------------------------------------
y_pred_scaled = model.predict(X_last).flatten()
# inverse scale for target only
data_min, data_max = scaler.data_min_[0], scaler.data_max_[0]
y_pred = y_pred_scaled * (data_max - data_min) + data_min
ny_true = y_last.flatten() * (data_max - data_min) + data_min

ds = df_feat.index[-N_FORECAST:]

# metrics
r2   = r2_score(ny_true, y_pred)
mae  = mean_absolute_error(ny_true, y_pred)
rmse = mean_squared_error(ny_true, y_pred, squared=False)
print(f"Forecast → R²={r2:.4f}  MAE={mae:.2f}  RMSE={rmse:.2f}")

# plot
plt.figure(figsize=(12,4))
plt.plot(ds, ny_true, '-o', label='Actual')
plt.plot(ds, y_pred, '-o', label='Forecast')
plt.title('Building Consumption LSTM Forecast – Last 24 h')
plt.xlabel('Time')
plt.ylabel('Consumption')
plt.legend()
plt.tight_layout()
plt.show()


c:\Users\user\enviroments\Stage_project\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/80
70/70 - 10s - 147ms/step - loss: 0.0181 - val_loss: 0.0161
Epoch 2/80
70/70 - 6s - 87ms/step - loss: 0.0097 - val_loss: 0.0107
Epoch 3/80
70/70 - 6s - 88ms/step - loss: 0.0076 - val_loss: 0.0084
Epoch 4/80
70/70 - 7s - 96ms/step - loss: 0.0066 - val_loss: 0.0068
Epoch 5/80
70/70 - 8s - 120ms/step - loss: 0.0060 - val_loss: 0.0063
Epoch 6/80
70/70 - 7s - 107ms/step - loss: 0.0055 - val_loss: 0.0060
Epoch 7/80
70/70 - 8s - 109ms/step - loss: 0.0052 - val_loss: 0.0062
Epoch 8/80
70/70 - 8s - 111ms/step - loss: 0.0050 - val_loss: 0.0063
Epoch 9/80
70/70 - 10s - 146ms/step - loss: 0.0047 - val_loss: 0.0061
Epoch 10/80
70/70 - 8s - 112ms/step - loss: 0.0045 - val_loss: 0.0061
Epoch 11/80
70/70 - 8s - 109ms/step - loss: 0.0043 - val_loss: 0.0058
Epoch 12/80
70/70 - 8s - 114ms/step - loss: 0.0041 - val_loss: 0.0051
Epoch 13/80
70/70 - 9s - 128ms/step - loss: 0.0040 - val_loss: 0.0054
Epoch 14/80
70/70 - 9s - 124ms/step - loss: 0.0039 - val_loss: 0.0051
Epoch 15/80
70/70 - 9s - 126ms

TypeError: got an unexpected keyword argument 'squared'